## Reference Materials

https://github.com/marcotcr/lime?tab=readme-ov-file

In [87]:
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
from __future__ import print_function


In [88]:
import pandas as pd
import argparse, os
import matplotlib.pyplot as plt
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch import nn
from torch.utils.data import DataLoader 

In [89]:
def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument("--layer", type=float, default=3)
    parser.add_argument("--hidden", type=int, default=128)
    parser.add_argument("--hidden1", type=int, default=64)
    parser.add_argument("--hidden2", type=int, default=32)
    parser.add_argument("--lr", type=float, default=1e-3)
    parser.add_argument("--seed", type=float, default=42)
    parser.add_argument("--epoch", type=int, default=50)
    
    return parser.parse_args()

In [90]:
class Q_vecDataset(torch.utils.data.Dataset):
  def __init__(self, X_rus, y_rus, train):
        self.feature = X_rus
        self.target = y_rus
        self.train = train

  def __len__(self):
        return self.target.shape[0]

  def __getitem__(self, index):
        X = self.feature.iloc[[index]]
        X = X.to_numpy()
        y = self.target.iloc[[index]]
        y= y.to_numpy()

        return X, y

In [91]:
def prepare_data():
    q_cleaned_old = pd.read_csv('data/data_vectorized_240228.csv')
    q_cleaned_old.drop(['ia_status_Facility Study', 'ia_status_Feasibility Study',
        'ia_status_IA Executed', 'ia_status_Operational',
        'ia_status_System Impact Study', 'Unnamed: 0'], axis = 1, inplace=True)

    exempt = []
    for col in list(q_cleaned_old.columns):
        if q_cleaned_old[col].max() < 1:
            exempt.append(col)
    q_cleaned_old.drop(columns = exempt, inplace=True)
    
    # Use batch normalization here - subtract by mean of data + divide by variance
    scaler = StandardScaler()
    scaler.fit(q_cleaned_old)
    q_cleaned_array = scaler.transform(q_cleaned_old)
    q_cleaned = pd.DataFrame(q_cleaned_array, columns=q_cleaned_old.columns)
    

    features = q_cleaned.drop(['ia_status_Withdrawn'], axis = 1)
    target = q_cleaned_old['ia_status_Withdrawn']

    seed = 42

    rus = RandomUnderSampler(random_state=seed)
    X_rus, y_rus= rus.fit_resample(features, target)
    X_train, X_test, y_train, y_test = train_test_split(X_rus, y_rus,
                                                            test_size = 0.2,
                                                            random_state = seed)
    return X_train, X_test, y_train, y_test

In [92]:
%cd git\Quennect

[WinError 3] 지정된 경로를 찾을 수 없습니다: 'git\\Quennect'
c:\Users\celin\git\Quennect


In [93]:
X_train, X_test, y_train, y_test = prepare_data()

In [94]:
X_train = np.array(X_train)

In [95]:
X_train

array([[-0.08696952, -0.11717451, -0.42817022, ..., -0.03576319,
        -0.02340396, -0.02340396],
       [-0.25977336,  1.02335176, -0.42817022, ..., -0.03576319,
        -0.02340396, -0.02340396],
       [ 0.9498535 ,  0.53455479, -0.42817022, ..., -0.03576319,
        -0.02340396, -0.02340396],
       ...,
       [ 0.08583431, -0.44303915, -0.42817022, ..., -0.03576319,
        -0.02340396, -0.02340396],
       [ 0.60424583,  0.53455479, -0.42817022, ..., -0.03576319,
        -0.02340396, -0.02340396],
       [ 0.25863815,  0.20869014, -0.42817022, ..., -0.03576319,
        -0.02340396, -0.02340396]])

In [114]:
X_test = np.array(X_test)

In [96]:
classes = np.array(['Not Withdrawn', 'Withdrawn'])

In [97]:
q_cleaned_old = pd.read_csv('data/data_vectorized_240228.csv')
q_cleaned_old.drop(['ia_status_Facility Study', 'ia_status_Feasibility Study',
        'ia_status_IA Executed', 'ia_status_Operational',
        'ia_status_System Impact Study', 'Unnamed: 0'], axis = 1, inplace=True)
q_cleaned_old.drop(['ia_status_Withdrawn'], axis = 1)

,q_year,prop_year,region_CAISO,region_ERCOT,region_ISO-NE,region_MISO,region_NYISO,region_PJM,region_SPP,region_Southeast (non-ISO),...,util_tsgt,util_ugi,util_unknown,util_upper peninsula power company,util_vea,util_wabash valley power,util_wisconsin electric power company,util_wisconsin public service corporation,"util_wolverine power supply cooperative, inc.",util_xcel energy
0,2016.0,2018.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,2008.0,2011.0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,2019.0,2023.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2009.0,2010.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2020.0,2023.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5475,2005.0,2007.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5476,2008.0,2010.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5477,2003.0,2006.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5478,2007.0,2007.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
feature_names = q_cleaned_old.columns

In [99]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, feature_names=feature_names, class_names=classes, discretize_continuous=True)

In [100]:
class NeuralNetwork(nn.Module): # nn.Module = base case for all neural network modules
    # we define model as a subclass of nn.Module -> it creates parameters of the modules with utility methods like eval()
        def __init__(self):
            super().__init__()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(140, int(128)), # apply linear transformation to the incoming data : y = x*W^T+b
                                        # weight here will be size of output * input
                nn.ReLU(),  # rectified linear unit function: 0 for values < 0 and linear function if > 0
                nn.Linear(int(128), int(64)),
                nn.ReLU(),
                nn.Linear(64, 32),
                nn.ReLU(),
                nn.Linear(32, 64),
                nn.ReLU(),
                nn.Linear(64, 2),
            )
            #self.sig = nn.Sigmoid() 
            #self.softmax = nn.Softmax(dim=1)
            # TODO: BCELoss does not expect raw logits - every value should be in the range [0,1].
            # TODO: Check what the previous model was doing, if there was regularization, learning rate, etc.
            
        def forward(self, x): 
            x = self.linear_relu_stack(x)
            #x = self.sig(x)
            #x = self.softmax(x)
            return x # changed to squeeze

In [101]:
filepath = os.path.join("model", "epoch5_lime.pt")
the_model = NeuralNetwork()
the_model = torch.load(filepath)

In [102]:
the_model.eval()

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=140, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=64, bias=True)
    (7): ReLU()
    (8): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [103]:
X_test

,q_year,prop_year,region_CAISO,region_MISO,region_PJM,region_Southeast (non-ISO),region_West (non-ISO),total_mw,type1_Battery,type1_Biofuel,...,util_tsgt,util_ugi,util_unknown,util_upper peninsula power company,util_vea,util_wabash valley power,util_wisconsin electric power company,util_wisconsin public service corporation,"util_wolverine power supply cooperative, inc.",util_xcel energy
5084,-0.778185,-1.094768,-0.42817,-0.521020,0.950494,-0.178384,-0.285496,-0.616488,-0.348342,-0.035763,...,-0.03022,-0.01351,-0.01351,-0.01351,-0.040559,-0.01351,-0.033107,-0.035763,-0.023404,-0.023404
1306,0.949854,0.534555,-0.42817,-0.521020,-1.052085,5.605893,-0.285496,-0.597235,2.870740,-0.035763,...,-0.03022,-0.01351,-0.01351,-0.01351,-0.040559,-0.01351,-0.033107,-0.035763,-0.023404,-0.023404
827,0.949854,0.860419,2.33552,-0.521020,-1.052085,-0.178384,-0.285496,0.034260,-0.348342,-0.035763,...,-0.03022,-0.01351,-0.01351,-0.01351,-0.040559,-0.01351,-0.033107,-0.035763,-0.023404,-0.023404
1808,-1.469400,-1.420633,-0.42817,-0.521020,0.950494,-0.178384,-0.285496,0.138225,-0.348342,-0.035763,...,-0.03022,-0.01351,-0.01351,-0.01351,-0.040559,-0.01351,-0.033107,-0.035763,-0.023404,-0.023404
1101,0.604246,0.534555,-0.42817,-0.521020,-1.052085,-0.178384,3.502680,-0.366200,-0.348342,-0.035763,...,-0.03022,-0.01351,-0.01351,-0.01351,-0.040559,-0.01351,-0.033107,-0.035763,-0.023404,-0.023404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
863,-1.123793,-1.257701,2.33552,-0.521020,-1.052085,-0.178384,-0.285496,-0.346947,-0.348342,-0.035763,...,-0.03022,-0.01351,-0.01351,-0.01351,-0.040559,-0.01351,-0.033107,-0.035763,-0.023404,-0.023404
3998,0.777050,1.023352,-0.42817,-0.521020,0.950494,-0.178384,-0.285496,-0.250683,-0.348342,-0.035763,...,-0.03022,-0.01351,-0.01351,-0.01351,-0.040559,-0.01351,-0.033107,-0.035763,-0.023404,-0.023404
5203,-2.851831,-2.561159,-0.42817,-0.521020,0.950494,-0.178384,-0.285496,0.057363,-0.348342,-0.035763,...,-0.03022,-0.01351,-0.01351,-0.01351,-0.040559,-0.01351,-0.033107,-0.035763,-0.023404,-0.023404
113,-0.605381,-1.257701,-0.42817,1.919313,-1.052085,-0.178384,-0.285496,-0.366200,-0.348342,-0.035763,...,-0.03022,-0.01351,-0.01351,-0.01351,-0.040559,-0.01351,-0.033107,-0.035763,-0.023404,-0.023404


In [116]:
# Define a function to convert input from NumPy arrays to PyTorch tensors
def numpy_to_tensor(x):
    return torch.tensor(x, dtype=torch.float32)

# Define a function to convert output from PyTorch tensors to NumPy arrays
def tensor_to_numpy(x):
    return x.detach().numpy()

# Wrap the forward pass of your model with NumPy-to-tensor and tensor-to-NumPy conversions
def model_forward_numpy(x_numpy):
    x_tensor = numpy_to_tensor(x_numpy)
    output_tensor = the_model(x_tensor)
    output_tensor = torch.nn.functional.softmax(output_tensor, dim=1)
    output_numpy = tensor_to_numpy(output_tensor)
    return output_numpy

In [117]:
#i = np.random.randint(0, X_test.shape[0])
exp = explainer.explain_instance(X_test[1], model_forward_numpy, num_features=2, top_labels=1, num_samples=1000)